In [1]:
import re
import os
import glob
import pickle
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from scipy.stats import zscore
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.nn.utils.rnn import pad_sequence
from torchvision import models

In [2]:
df = pd.read_pickle("data/train.pkl")

In [3]:
def pad_collate(batch):
    samples, labels = zip(*((sample.transpose(1, 0), label)
                            for (sample, label, *_) in batch))
    samples = pad_sequence(
        samples, batch_first=True).transpose(1, 2).contiguous()
    return samples, torch.stack(labels)


class AtrialStenosisDataset(Dataset):
    def __init__(self, data_dir): # data_origin -> fnkv other option
        """
        data_dir: data path
        """
        self.data_dir = pickle.load(open(data_dir,'rb'))#pd.read_pickle(data_dir)
        
   
    def __len__(self):
        return len(self.data_dir)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        data = pd.read_csv(self.data_dir.iloc[idx].ecg_path)
        data = data.drop(" ", 1)
        data = data.to_numpy()
        
        input = data

        input = input.astype(np.float32)
        input = zscore(input, axis=-1)         
        input = torch.from_numpy(input)
    
        anot = self.data_dir.iloc[idx]["as"]
        
        return data, anot

In [29]:
train_dataset = AtrialStenosisDataset( data_dir = 'data/train.pkl')
trainloader = DataLoader(train_dataset, batch_size=1, num_workers=4, shuffle=True, drop_last=True)

valid_dataset = AtrialStenosisDataset( data_dir = 'data/test.pkl')
validloader = DataLoader(valid_dataset, batch_size=1, num_workers=4, shuffle=False, drop_last=True)

In [ ]:
data, anot = next(iter(trainloader))
print(data.shape)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'AtrialStenosisDataset' on <module '__main__' (built-in)>


In [ ]:
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Running on {device}..")

In [7]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.cnn1 = models.resnet18(pretrained=False)
        self.fc1 = nn.Linear(2*1000, 2)


    def forward(self, input1):
        cnn_out1 = self.cnn1(input1)
        output = self.fc1(cnn_out1)
        
        return output

In [28]:
from resnet import resnet50

model = resnet50()
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)

epochs = 50

w_0 = 1.531
w_1 = 8.576

m = nn.Softmax(dim=1)

class_weights = torch.FloatTensor([w_0, w_1]).to(device)
criterion = nn.CrossEntropyLoss(class_weights)

training_loss = []
validation_loss = []

for epoch in range(1, epochs):
    
    model.train()
    running_loss = 0
    running_loss_validation = 0
    
    for i,[data, anot] in enumerate(trainloader):
        input = data.to(device=device).float()
        anot = anot.to(device=device)
        
        input = input.permute(0, 2, 1)  
        
        if anot == 1:
            in2 = input[:, :, 0:2500]
            in2 = input[:, :, 2500:]
            
            for input in [in1 ,in2]:
                
                output = model(input)

                optimizer.zero_grad()

                loss = criterion(output, anot)
                loss_item = loss.item()
                loss.backward()
                optimizer.step()
                running_loss += loss_item
                
                print(f"{epoch}: {i}/{len(trainloader)} - loss: {loss_item}, class: {anot}")
        else:
            output = model(input)
            optimizer.zero_grad()
            loss = criterion(output, anot)
            loss_item = loss.item()
            loss.backward()
            optimizer.step()
            running_loss += loss_item
        
            print(f"{epoch}: {i}/{len(trainloader)} - loss: {loss_item}, class: {anot}")
        
    loss_sum = running_loss/(i)
    training_loss.append(loss_sum)
    
    loss_sum_valid = running_loss_validation/(j)
    validation_loss.append(loss_sum_valid)
    
    torch.save(model.state_dict(),"mymodels/weights/2Resnet"+str(epoch)+".pt") #ukladanie modelu po každej epoche
    #learnin curve
    fig = plt.figure()
    plt.plot(training_loss, label="training loss")
    plt.plot(validation_loss, label="validation loss")
    plt.title("Training loss ")
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.savefig("2Training_loss.png")
    plt.close()

/var/folders/qz/tnst848s0vz2c7b4sf96rql40000gn/T/ipykernel_54657/3765501210.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(" ", 1)


1: 0/12727 - loss: 1.491615891456604, class: tensor([1])
1: 0/12727 - loss: 1.0623211860656738, class: tensor([1])
1: 1/12727 - loss: 0.6662098169326782, class: tensor([0])


KeyboardInterrupt: 

In [26]:
len(validloader)

3270

In [21]:
in1 = input[:, :, 0:2500]
in2 = input[:, :, 2500:]
print(in1.shape)
print(in2.shape)

torch.Size([1, 12, 2500])
torch.Size([1, 12, 2500])


In [ ]:
def my_collate(batch):
    image = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return image, targets